In [1]:
import numpy as np
import pandas as pd

# Загрузка тренировочных данных

In [8]:
data_train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=0, names=['text', 'label'])

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [14]:
texts = list(data_train['text'].to_numpy())

In [18]:
labels = list(data_train['label'].to_numpy())

## Подготовим токенизатор и переведем текста в матричное представление

In [15]:
vec = TfidfVectorizer(texts)

In [16]:
X_train = vec.fit_transform(texts)

In [17]:
X_train.shape

(1999, 3973)

# Выбор классификатора

In [19]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(cross_val_score(clf(max_iter=1000), X_train, labels).mean())
    print("\n")

0.7703922305764411


0.7673784461152883


0.7543684210526316




# Добавим биграммы в представление

In [67]:
vec2 = TfidfVectorizer(ngram_range=(1,2))

In [68]:
X_train2 = vec2.fit_transform(texts)

In [69]:
X_train2.shape

(1999, 22781)

Снова проверяем классификаторы

In [73]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(clf(max_iter=10000), X_train2, labels).mean())

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.7093596491228069
<class 'sklearn.svm._classes.LinearSVC'>
0.7838834586466165
<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.7803859649122808


За основной классификатор выбираю LinearSVC

In [115]:
clf = LinearSVC().fit(X_train2, labels)

In [113]:
data_test = pd.read_csv('products_sentiment_test.tsv', sep='\t', header=0, names=['ids', 'texts'])

In [114]:
data_test.head()

,ids,texts
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [116]:
X_text_test = list(data_test['texts'].to_numpy())

In [117]:
X_test = vec2.transform(X_text_test)

In [118]:
X_test.shape

(500, 22781)

In [120]:
preds = clf.predict(X_test)

In [121]:
ans = pd.DataFrame({'Id' : np.arange(0, len(preds)), 'y' : np.array(preds)})

In [123]:
ans.to_csv('submission.csv', index=False)